In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage import morphology,color,io
from skimage.morphology import binary_dilation
from skimage.color import rgb2gray
from sklearn.cluster import KMeans

In [ ]:
def segmentation(img, n_clusters, im_type="rgb"):
    im_raw = np.reshape(img,(img.shape[0]*img.shape[1],img.shape[2]))
    #K_means
    kmeans = KMeans(n_clusters=n_clusters).fit(im_raw)
    im_seg = np.reshape(kmeans.labels_,(img.shape[0],img.shape[1]))
    new_img = np.zeros((img.shape[0],img.shape[1],3))
    #Asign center
    for i,j in zip(kmeans.cluster_centers_, range(len(kmeans.cluster_centers_))):
        new_img[im_seg == j] = i
        new_img = new_img.astype(np.uint8)
    return(new_img)

In [ ]:
contenido = os.listdir('/content/drive/MyDrive/Images2')
out_path='/content/drive/MyDrive/Images_process/'
struct = morphology.disk(21)
for carpeta in contenido:
    carpeta1=os.listdir('/content/drive/MyDrive/Images2/'+str(carpeta))
    for subcarpeta in carpeta1:
        subcarpeta1=os.listdir('/content/drive/MyDrive/Images2/'+str(carpeta)+'/'+str(subcarpeta))
        for archivo in subcarpeta1:
            in_path = '/content/drive/MyDrive/Images2/'+str(carpeta)+'/'+str(subcarpeta)+'/'+str(archivo)
            print(in_path)
            im_ori=io.imread(in_path)
            md_im = ndimage.median_filter(im_ori, size=5)
            mask=np.copy(md_im[:,:,0])
            mglob=np.min(mask)
            sglob=np.std(mask)
            mask[mask<=mglob+sglob]=255
            mask[mask<255]=1
            mask[mask==255]=0
            im_seg=np.copy(im_ori)
            im_seg[:,:,0]=im_seg[:,:,0]*mask
            im_seg[:,:,1]=im_seg[:,:,1]*mask
            im_seg[:,:,2]=im_seg[:,:,2]*mask
            im_seg[im_seg==0]=255
            n_clusters = 4
            im_knn = np.expand_dims(im_seg, axis=-1)
            im_knn = np.repeat(im_seg, 3, axis=-1)
            im_knn = segmentation(im_seg, n_clusters)
            gimg=rgb2gray(im_knn)
            min_knn=np.ones(gimg.shape)
            min_knn[im_knn[:,:,0]<=np.min(im_knn[:,:,0])]=0#
            op_im = morphology.opening(min_knn, struct)
            op_im=np.abs(op_im-1)
            md_imk = ndimage.median_filter(op_im, size=60)
            im_fin=np.copy(im_ori)
            im_fin[:,:,0]=im_fin[:,:,0]*md_imk
            im_fin[:,:,1]=im_fin[:,:,1]*md_imk
            im_fin[:,:,2]=im_fin[:,:,2]*md_imk
            im_fin[im_fin==0]=255
            io.imsave(str(out_path)+str(archivo),im_fin)

/content/drive/MyDrive/Images2/C5/C5_8/C5_8_2_b.jpg
/content/drive/MyDrive/Images2/C5/C5_8/C5_8_3_b.jpg
/content/drive/MyDrive/Images2/C5/C5_8/C5_8_1.jpg
/content/drive/MyDrive/Images2/C5/C5_8/C5_8_3_a_f.jpg
/content/drive/MyDrive/Images2/C5/C5_8/C5_8_2_a.jpg
/content/drive/MyDrive/Images2/C5/C5_8/C5_8_3_b_f.jpg
/content/drive/MyDrive/Images2/C5/C5_8/C5_8_2_b_f.jpg
/content/drive/MyDrive/Images2/C5/C5_8/C5_8_2_a_f.jpg
/content/drive/MyDrive/Images2/C5/C5_8/C5_8_1_f.jpg
